<a href="https://colab.research.google.com/github/erberry/ThinkML/blob/main/classifier_mlp_debug_stepbystep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 解析多层感知机的代码实现，一步步认识神经网络的学习过程

多层感知机是最基础的神经网络之一，本文使用sklearn包中的MLPClassifier类，来学习基础的神经网络的学习过程。

### 熟悉数据集

为了更好的理解，我们先熟悉一下神经网络学习所使用的数据集

训练数据X_train的大小为(60,2),既60条数据，每条数据有两个特征，可以想象为平面坐标系中的坐标，如下图：

![abc](https://drive.usercontent.google.com/download?id=1gUW7tXdfRn1lkF-BVvN0Kajgi5j8F8Ic&export=view&authuser=0)

数据如下：

```
array([[ 0.77455385,  0.15375803],
       [ 0.06873258,  0.56648467],
       [ 0.95217454, -0.75307471],
       [ 1.69945309,  0.58771967],
       [ 2.18137168, -0.02291747],
       [ 1.16339832,  0.55290238],
       [-0.56175303,  1.05486051],
       [-0.74662946, -0.3829632 ],
       [ 0.20718083, -0.09767143],
       [ 1.22540835,  1.19793017],
       [-0.65805008, -0.12944211],
       [-0.0551441 , -0.03733246],
       [ 0.30430746,  0.82373935],
       [-0.25451559,  0.19317272],
	   ...])
```



标签y_train的大小为(60,1)，既60条数据，和X_train数据一一对应，标记了X_train数据的分类：0或者1，上图中的红色和蓝色。

```
array([0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1])
```

class MLPClassifier 是 Multi-layer Perceptron（多层感知机）在sklearn中的实现类。使用如上的数据集对 MLPClassifier 进行训练，将使得它得到数据集的分类知识，如下图，红色区域代表模型认为这个区域所有的点应该分类到A类，蓝色区域代表模型认为这个区域所有的点应该分类到B类,中间浅色则代表模型认为这里的点属于有概率属于A类或者B类，颜色的深浅代表了概率的高低。

![abc](https://drive.usercontent.google.com/download?id=1hXRdu3NACZnTBCAdq933hOH6_JmazeZh&export=view&authuser=0)

MLPClassifier 继承自ClassifierMixin 和 BaseMultilayerPerceptron，其中训练和推理的计算大部分都在基类 BaseMultilayerPerceptron 中实现。

MLPClassifier 的构造函数的第一个参数 hidden_layer_sizes 指定了隐藏层的大小，默认为只有一层隐藏层，且大小为100，既这一层有100个神经节点。如果设置 hidden_layer_sizes = (100,50,200),代表隐藏层有三层，第一层有100个节点，第二层有50个节点，第三层有200个节点。



```
class MLPClassifier(ClassifierMixin, BaseMultilayerPerceptron):
    """Multi-layer Perceptron classifier.

    This model optimizes the log-loss function using LBFGS or stochastic
    gradient descent.

    .. versionadded:: 0.18

    Parameters
    ----------
    hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
        The ith element represents the number of neurons in the ith
        hidden layer.
```




fit(model fitting,模型拟合或模型训练) 使用训练数据X_train和标签y_train来训练模型。

fit方法会根据输入的特征和目标值调整模型的参数。
```
clf.fit(X_train, y_train)
```




### 初始化感知机的神经网络节点

在进行训练之前，需要对神经网络进行初始化。包括层数的初始化、参数的初始化（权重和偏置）、激活值列表、误差列表、权重参数梯度列表、偏置参数梯度列表

多层感知机的神经网络由：输入层、隐藏层、输出层以及层之间的参数（权重和偏置）组成，结构示意图如下：

- 输入层：输入的训练数据为 (60X2) 的矩阵，其中60可以是任意值，它是一次批量处理的数量；2为数据的特征数量。
- 输出层：输出的结果为(60X1)，其中60对应输入的批量数量；1代表它是一个binary classifier，输入数据被二分类。
- 隐藏层：大小为(100,)，既一个隐藏层，大小为100个节点

![abc](https://drive.usercontent.google.com/download?id=1oTJMCq2IthYiuWuoNxsIvoM1I89Xpkiy&export=view&authuser=0)

如下代码，在训练开始之前，初始化了层数信息、每层对应的权重和偏置矩阵



```
class BaseMultilayerPerceptron(BaseEstimator, metaclass=ABCMeta):

    def _fit(self, X, y, incremental=False):
        # 隐藏层大小，从类构造函数传入，默认为(100,)，代表只有一层，节点数为100
        # hidden_layer_sizes = [100]
        hidden_layer_sizes = self.hidden_layer_sizes
        
        ...
        # 训练数据的大小，n_samples代表批量处理多少条样本，n_features代表每条样本有几个特征，本次示例为(60,2),n_samples=60,n_features=2
        n_samples, n_features = X.shape

        ...

        # 标签数据大小，因为是二分类问题，一个值（0或1）就可以代表输入的分类结果
        # self.n_outputs_=1
        self.n_outputs_ = y.shape[1]

        # 神经网络形状，见上图
        # layer_units = [2,100,1]
        layer_units = [n_features] + hidden_layer_sizes + [self.n_outputs_]

        ...

        # 如果是首次，调用_initialize，初始化权重和偏置
        # X.dtype为输入数据的数据类型，如float32、float64等
        if first_pass:
            # First time training the model
            self._initialize(y, layer_units, X.dtype)

        # Initialize lists
        activations = [X] + [None] * (len(layer_units) - 1)
        deltas = [None] * (len(activations) - 1)

        coef_grads = [
            np.empty((n_fan_in_, n_fan_out_), dtype=X.dtype)
            for n_fan_in_, n_fan_out_ in zip(layer_units[:-1], layer_units[1:])
        ]

        intercept_grads = [
            np.empty(n_fan_out_, dtype=X.dtype) for n_fan_out_ in layer_units[1:]
        ]

        ...

    def _initialize(self, y, layer_units, dtype):
        self.n_iter_ = 0
        self.t_ = 0
        self.n_outputs_ = y.shape[1]

        # Compute the number of layers
        # layer_units = [2,100,1]
        # self.n_layers_ = 3
        self.n_layers_ = len(layer_units)

        # 选择输出层激活函数，本次是二分类问题，选择logistic
        if not is_classifier(self):
            self.out_activation_ = "identity"
        # Output for multi class
        elif self._label_binarizer.y_type_ == "multiclass":
            self.out_activation_ = "softmax"
        # Output for binary class and multi-label
        else:
            self.out_activation_ = "logistic"

        # 初始化权重和偏置变量
        # coefs_：coefficient，系数，权重
        # intercepts_：intercept，截距，偏置
        self.coefs_ = []
        self.intercepts_ = []

        # 对每一层的参数进行初始化
        # 第0和1层之间的参数 self.coefs_[0] 的大小为(2,100) self.intercepts_[0] 的大小为100
        # 第1和2层之间的参数 self.coefs_[1] 的大小为(100,1) self.intercepts_[1] 的大小为1
        for i in range(self.n_layers_ - 1):
            coef_init, intercept_init = self._init_coef(
                layer_units[i], layer_units[i + 1], dtype
            )
            self.coefs_.append(coef_init)
            self.intercepts_.append(intercept_init)

        ...
```



在初始化权重、偏置完成后，回到_fit方法，继续对激活值列表、误差列表、权重参数梯度列表、偏置参数梯度列表进行初始化



```
class BaseMultilayerPerceptron(BaseEstimator, metaclass=ABCMeta):

    def _fit(self, X, y, incremental=False):
        ...

        # 初始化激活值列表
        # activations = [array([[ 0.43637123, -0.11525971],...]), None, None]
        # 后续训练中会用于存储输入层、隐藏层、输出层的结果 [X,Y,T]
        activations = [X] + [None] * (len(layer_units) - 1)
        # 初始化误差列表
        # deltas = [None, None]
        # 后续训练中会用于存储隐藏层的误差，输出层的误差
        deltas = [None] * (len(activations) - 1)

        # 初始化权重参数梯度列表,用于训练过程中存储权重的梯度
        # coef_grads[0].shape = (2,100) 第一层权重的梯度，大小和第一层权重一致
        # coef_grads[1].shape = (100,1) 第二层权重的梯度，大小和第二层权重一致
        coef_grads = [
            np.empty((n_fan_in_, n_fan_out_), dtype=X.dtype)
            for n_fan_in_, n_fan_out_ in zip(layer_units[:-1], layer_units[1:])
        ]

        # 初始化偏置参数梯度列表，用于训练过程中存储偏置的梯度
        # intercept_grads[0].shape = (100,) 第一层偏置的梯度，大小和第一层偏置一致
        # intercept_grads[1].shape = (1,) 第二层偏置的梯度，大小和第二层偏置一致
        intercept_grads = [
            np.empty(n_fan_out_, dtype=X.dtype) for n_fan_out_ in layer_units[1:]
        ]

        ...
```



回顾一下对哪些参数进行了初始化：
- self.coefs_：权重
- self.intercepts_：偏置
- activations：激活值列表
- deltas：误差
- coef_grads：权重梯度
- intercept_grads：偏置梯度

这些参数是正向推理和反向传播过程中的核心参数。

### 正向传播

神经网络的学习过程包括了正向传播和反向传播过程，其目的是计算出每一层权重和偏置参数的更新值，并对参数进行更新，最终提高模型的准确性。

其中正向传播是使用当前的权重和偏置参数对输入进行预测的过程，最终得到一个预测结果。

回顾上图，图中只包含了矩阵计算，实际每一层计算的结果还需要经过激活函数进行一次转换。本次代码中隐藏层和输出层的激活函数分别为 relu 和 logsitic（同sigmod）

- 隐藏层激活函数 relu： f(x) = max(0, x)
- 输出层激活函数 logistic： f(x) = 1 / (1 + exp(-x))

每一层的计算都可以抽象为：首先进行一次矩阵计算，再使用激活函数进行一次转换，表达如下：

`z = w*x+b ; a = activation_func(z)`

为了评估神经网络的准确性，需要先进行一次正向传播，输出神经网络的预测，反向传播使用真实值和预测值计算预测的准确性。

如下代码展示了在进行反向传播之前进行的正向传播的逻辑，其中关键变量如下：

- self.coefs_：权重，用b作为简称
- self.intercepts_：偏置，用w作为简称
- activations：激活值列表，用a作为简称

以下代码逻辑用函数表达如下(z作为中间临时变量)：

- 输入层到隐藏层的计算： `z[0] = w[0] * x + b[0]` ; `a[0] = relu(z[0])`
- 隐藏层到输出层的计算： `z[1] = w[1] * a[0]  + b[1]` ; `a[1] = logistic(z[1])`


```
class BaseMultilayerPerceptron(BaseEstimator, metaclass=ABCMeta):

    def _backprop(self, X, y, activations, deltas, coef_grads, intercept_grads):

        n_samples = X.shape[0]

        # 正向传播
        # activations 缓存了每一层的计算结果
        # 本次只有三层，activations的值为 [输入值, 隐藏层计算结果，输出层计算结果]
        activations = self._forward_pass(activations)

        # 反向传播
        ...

    def _forward_pass(self, activations):
        for i in range(self.n_layers_ - 1):
            # w*x
            activations[i + 1] = safe_sparse_dot(activations[i], self.coefs_[i])
            # +b
            activations[i + 1] += self.intercepts_[i]

            # 如果不是最后一层，使用隐藏层激活函数对结果进行转换，本次是 relu
            if (i + 1) != (self.n_layers_ - 1):
                hidden_activation(activations[i + 1])

        # 最后一层激活函数，本次是 logistic
        output_activation = ACTIVATIONS[self.out_activation_]
        output_activation(activations[i + 1])

        return activations
```



正向传播过程中保存了每一层的中间结果：[输入值, 隐藏层计算结果，输出层计算结果]，输出层计算结果就是模型对输入的预测结果，用于反向传播前计算误差（理解为预测值和真实值之间的差距）。

### 反向传播

反向传播是神经网络学习中最常用的方法，它的核心是通过对损失函数求导数，使用导数做指引更新参数（权重和偏置），达到降低损失值（提高准确率）的目的。

那么如何使用导数降低损失值，下面一步步推导：


神经网络的学习结果为：得到合适的参数权重和偏置，使得神经网络能够得到准确的输出。

换句话说，神经网络的学习过程是：为了得到准确的输出，如何逐步调整权重和偏置的过程。

为了得到准确的输出，也就是提升神经网络的准确度，就是降低神经网络的误差。

神经网络的误差可以使用专用的损失函数对预测值和真实值进行评估，设损失函数为loss，预测值为a，真实值为y，则损失值为loss(a,y)。

回顾正向传播过程，预测值的计算过程为 `z = w*x+b ; a = activation_func(z)`

L = loss(activation_func(w*x+b),y)

由于y是常量，损失函数可以简单表示为关于自变量w和b的函数：L = loss(w,b)。问题转换为如何变化w和b，使得损失值L逐渐减小。



在神经网络的学习中，寻找最优参数（权重和偏置）时，要寻找使损失函数的值尽可能小的参数。为了找到使损失函数的值尽可能小的地方，需要计算参数的导数（确切地讲是梯度），然后以这个导数为指引，逐步更新参数的值。

**回顾使用导数找函数的最值**

导数是函数在某一点上的斜率，也可以理解为y相对于x的变化率（变化方向和变化速度）。

如下图，线性函数 y=x 导数为1，代表y和x是正相关的，x变大，y也变大，变化速度为1，例如：x从1变到2，y也从1变到2；x从1变到0，y也从1变到0

![abc](https://drive.usercontent.google.com/download?id=1FaEnUzEHwUdtpE2zf5l6EhaR1WEtsXio&export=view&authuser=0)

相反 y=-1 导数为-1，代表了y和x是负相关的

![abc](https://drive.usercontent.google.com/download?id=1OjEpk-xQtRqw-fihSVwgmjXe3BEEozLG&export=view&authuser=0)

也就是说在一个函数上，导数代表了x变化时，y将如何变化。

如果我们想要让y变小
- 当导数是正数时，代表y和x正相关，应该让x变小，y就会变小
- 当导数是负数时，代表y和x负相关，应该让x变大，y才会变小

当函数为 y=x*x+1 时，导数 y' = 2x

![abc](https://drive.usercontent.google.com/download?id=1fePZbdFv5pAbmDViJrB9ZZXiwhFc7MK8&export=view&authuser=0)

想要y变小
- 当x等于1时，导数为2，代表在x=1这个点上y和x正相关，应该让x变小，y就会变小
- 当x等于-1时，导数为-2，代表在x=-1这个点上y和x负相关，应该让x变大，y就会变小


假设有一个神经网络，现在我们来关注这个神经网络中的某一个权重参数。此时，对该权重参数的损失函数求导，表示的是“如果稍微改变这个权重参数的值，损失函数的值会如何变化”。如果导数的值为负，通过使该权重参数向正方向改变，可以减小损失函数的值；反过来，如果导数的值为正，则通过使该权重参数向负方向改变，可以减小损失函数的值。不过，当导数的值为 0 时，无论权重参数向哪个方向变化，损失函数的值都不会改变，此时该权重参数的更新会停在此处。

**损失函数**

明确了导数可以指引我们更新参数使得损失值变小后，我们回头来看损失函数。

神经网络的误差可以使用专用的损失函数对预测值和真实值进行评估，设损失函数为loss，预测值为a，真实值为y，则损失值为关于a和y的函数，且y为常量，则损失函数可以表示为如下的复合函数：

回顾正向传播过程，预测值的计算过程为 z = w*x+b ; a = activation_func(z)

L = loss(activation_func(w*x+b))



**回顾链式法则**

链式法则用于计算复合函数的导数。它说明了如何对由两个或多个函数复合而成的函数求导。

假设我们有两个函数 f 和 g，并且它们的复合函数是 h(x) = f(g(x)),那么根据链式法则，函数h(x)的导数可以表示为： h'(x) = f'(g(x)) * g'(x)。

举例，对应函数 y = sin(3x) 应用链式法则，导数为 y' = cos(3x) * 3

那么损失函数的导数就是分别对loss、activation_func、w*x+b 求导，并相乘。

**对损失函数求导，计算参数的梯度**


_backprop 方法使用反向传播算法和链式法则，计算参数梯度，具体代码解读如下：



```
    def _backprop(self, X, y, activations, deltas, coef_grads, intercept_grads):

        n_samples = X.shape[0]

        # 正向传播，得出预测值
        activations = self._forward_pass(activations)

        # 选择损失函数，本次任务是二分类任务，选择了 binary_log_loss
        loss_func_name = self.loss
        if loss_func_name == "log_loss" and self.out_activation_ == "logistic":
            loss_func_name = "binary_log_loss"

		# 传入真实值和预测值，计算损失值
        loss = LOSS_FUNCTIONS[loss_func_name](y, activations[-1])

        # 正则化相关，提升泛化能力
        values = 0
        for s in self.coefs_:
            s = s.ravel()
            values += np.dot(s, s)
        loss += (0.5 * self.alpha) * values / n_samples

        # 反向传播，self.n_layers_=3，层下标分别为 [0,1,2]
		# 最后一层为输出层，从倒数第二层开始，last = 1
        last = self.n_layers_ - 2

        # 求从隐藏层到输出层的梯度
		# 正向函数为 binary_log_loss(y, sigmod(activations[1] * w + b))
		# 应用链式法则，结果为 binary_log_loss' * sigmod' * (activations[1] * w + b)'
		# 其中 binary_log_loss' * sigmod' 计算并抵消后结果为 activations[-1] - y，暂存于 deltas[last]
        deltas[last] = activations[-1] - y

        # 调用_compute_loss_grad计算隐藏层与输出层之间的参数（权重和偏置）的梯度
        self._compute_loss_grad(
            last, n_samples, activations, deltas, coef_grads, intercept_grads
        )

		# 根据使用的激活函数，获取对应的求导方法，本次隐藏层激活函数为 relu
        inplace_derivative = DERIVATIVES[self.activation]

        # 计算隐藏层的梯度
		# 如果隐藏层有多层，因为正向函数相同，可以循环处理
        for i in range(self.n_layers_ - 2, 0, -1):
			# 正向函数为 binary_log_loss(y, sigmod(a * self.coefs_[1] + self.intercept_[1])); a = relu(activations[0] * w + b)
			# 应用链式法则，结果为 binary_log_loss' * sigmod' * (a * self.coefs_[1] + self.intercept_[1]))' * relu' * (activations[0] * w + b)'
			# binary_log_loss' * sigmod' 结果为 deltas[1]，(a * self.coefs_[1] + self.intercept_[1]))' 的导数为 self.coefs_[1]
			# deltas[0] = deltas[1] * self.coefs_[1]
            deltas[i - 1] = safe_sparse_dot(deltas[i], self.coefs_[i].T)
			# 继续对激活函数relu求导，并应用链式法则，结果存入deltas[0]
            inplace_derivative(activations[i], deltas[i - 1])

			# 计算梯度，同上
            self._compute_loss_grad(
                i - 1, n_samples, activations, deltas, coef_grads, intercept_grads
            )

		# 返回参数的梯度
        return loss, coef_grads, intercept_grads


	# 用于计算 activations[layer] * w + b 的导数，并计算梯度
	# 求得的梯度结果分别存储于 coef_grads intercept_grads
    def _compute_loss_grad(
        self, layer, n_samples, activations, deltas, coef_grads, intercept_grads
    ):

		# activations[layer] * w + b 对 w 求偏导，值为 activations[layer]
		# 再乘以前面损失函数和激活函数的导数
        coef_grads[layer] = safe_sparse_dot(activations[layer].T, deltas[layer])
		# 正则化相关，提升泛化能力
        coef_grads[layer] += self.alpha * self.coefs_[layer]
		# 由于是批量处理，要除以批量的样本数，求均值
        coef_grads[layer] /= n_samples

		# activations[layer] * w + b 对 b 求偏导，值为 1
		# 再乘以前面损失函数和激活函数的导数
		# 由于是批量处理，求均值
        intercept_grads[layer] = np.mean(deltas[layer], 0)
```



**更新梯度**

在获得参数的梯度后，就可以乘以学习率，对参数进行更新了。

最基本的更新计算步骤为 `update = learning_rate * (-grad); param += update` 之所以grad取反，是因为我们的目的是损失值降低，如果grad为正，代表参数和损失值正相关，参数应该变小；如果grad为负，代表负相关，参数应该变大。

但这样恒定的学习率，在一些复杂场景下，会导致模型不能收敛或者收敛震荡以及收敛慢等问题。所以模型大部分都是使用优化器，引入了动量、自适应学习率等机制来改进训练效果。这里不再展开，更新参数的代码如下：



```
 params = self.coefs_ + self.intercepts_

  # update weights
 grads = coef_grads + intercept_grads
 self._optimizer.update_params(params, grads)
```


```
def update_params(self, params, grads):
    """Update parameters with given gradients

    Parameters
    ----------
    params : list of length = len(coefs_) + len(intercepts_)
        The concatenated list containing coefs_ and intercepts_ in MLP
        model. Used for initializing velocities and updating params

    grads : list of length = len(params)
        Containing gradients with respect to coefs_ and intercepts_ in MLP
        model. So length should be aligned with params
    """
    updates = self._get_updates(grads)
    for param, update in zip((p for p in params), updates):
        param += update
```



```
class AdamOptimizer(BaseOptimizer):
    """Stochastic gradient descent optimizer with Adam

    Note: All default values are from the original Adam paper

    Parameters
    ----------
    params : list, length = len(coefs_) + len(intercepts_)
        The concatenated list containing coefs_ and intercepts_ in MLP model.
        Used for initializing velocities and updating params

    learning_rate_init : float, default=0.001
        The initial learning rate used. It controls the step-size in updating
        the weights

    beta_1 : float, default=0.9
        Exponential decay rate for estimates of first moment vector, should be
        in [0, 1)

    beta_2 : float, default=0.999
        Exponential decay rate for estimates of second moment vector, should be
        in [0, 1)

    epsilon : float, default=1e-8
        Value for numerical stability

    Attributes
    ----------
    learning_rate : float
        The current learning rate

    t : int
        Timestep

    ms : list, length = len(params)
        First moment vectors

    vs : list, length = len(params)
        Second moment vectors

    References
    ----------
    :arxiv:`Kingma, Diederik, and Jimmy Ba (2014) "Adam: A method for
        stochastic optimization." <1412.6980>
    """

    def __init__(
        self, params, learning_rate_init=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8
    ):
        super().__init__(learning_rate_init)

        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.t = 0
        self.ms = [np.zeros_like(param) for param in params]
        self.vs = [np.zeros_like(param) for param in params]

    def _get_updates(self, grads):
        """Get the values used to update params with given gradients

        Parameters
        ----------
        grads : list, length = len(coefs_) + len(intercepts_)
            Containing gradients with respect to coefs_ and intercepts_ in MLP
            model. So length should be aligned with params

        Returns
        -------
        updates : list, length = len(grads)
            The values to add to params
        """
        self.t += 1
        self.ms = [
            self.beta_1 * m + (1 - self.beta_1) * grad
            for m, grad in zip(self.ms, grads)
        ]
        self.vs = [
            self.beta_2 * v + (1 - self.beta_2) * (grad**2)
            for v, grad in zip(self.vs, grads)
        ]
        self.learning_rate = (
            self.learning_rate_init
            * np.sqrt(1 - self.beta_2**self.t)
            / (1 - self.beta_1**self.t)
        )
        updates = [
            -self.learning_rate * m / (np.sqrt(v) + self.epsilon)
            for m, v in zip(self.ms, self.vs)
        ]
        return updates
```



